환경설정
API KEY를 설정

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API 키 정보 로드
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)

splits = text_splitter.split_documents(docs)
len(splits)

3

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [14]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# HuggingFace Model implementation
# repo_id is a registered model inside HuggingFace
repo_id = "google/gemma-2-9b-it"

llm_endpoint = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_new_tokens=512,
    temperature=0.1,
    huggingfacehub_api_token=hf_token
)

chat_llm = ChatHuggingFace(llm=llm_endpoint)

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt implementation
template = """당신은 질문-답변을 도와주는 AI 어시스턴트입니다. 
아래의 제공된 문맥(Context)을 활용해서만 질문에 답하세요. 
답을 모른다면 모른다고 말하고, 직접적인 답이 문맥에 없다면 문맥을 바탕으로 추론하지 마세요.
답변은 반드시 한국어로 작성하세요.

#Context:
{context}

#Question:
{question}

#Answer:"""

prompt = ChatPromptTemplate.from_template(template)

In [17]:
# 3. RAG 체인 생성
# 검색기(retriever) -> 컨텍스트 전달 -> 프롬프트 적용 -> 모델 실행 -> 결과 파싱
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_llm
    | StrOutputParser()
)

In [18]:
# 4. 실행 테스트
question = "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
response = rag_chain.invoke(question)

print(f"질문: {question}")
print("-" * 30)
print(f"답변: {response}")

질문: 부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?
------------------------------
답변: 부영그룹은 출산 직원에게 1억원을 지원합니다. 



In [19]:
from langchain_teddynote.messages import stream_response

In [20]:
question = "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
answer = rag_chain.invoke(question)

stream_response(answer)

부영그룹은 출산 직원에게 1억원을 지원합니다. 
